In [81]:
from glob import glob
from math import exp
import pandas as pd
import numpy as np
import os,sys
from bioinfokit import analys, visuz
from adjustText import adjust_text

os.chdir('/Users/jplab/Desktop/2021-7/data/7-5')

In [59]:
# vocano need p-value, match with deg data
deg_expr = pd.read_csv("/Users/jplab/Desktop/Projects/RNA_seq_workflow/downstream/deg_2021/DEG_B14_TGFb_ALL_filtered_expr.tsv",sep='\t')


# peak annotate data
anno_data = pd.read_csv('/Users/jplab/Desktop/2021-7/data/7-1/foxa1/test.B14_FOXA1.ATACUniq.K27ac-ann10000kb.closestfasle.tsv',sep='\t')
# print(anno_data.head())

# merge 
anno_with_expr = pd.merge(
    anno_data,
    deg_expr,
    # left_on='Nearest Ensembl',
    left_on='ensg',
    # right_on='ensg', 
    right_on='geneid', 
    how='outer')
anno_with_expr.head()
# print(anno_with_expr)  


not_null = anno_with_expr[anno_with_expr['ensg'].notnull()]
not_null = not_null[not_null['mean_B14_2.0'].notnull()]
not_null = not_null[not_null['geneid'].notnull()]
# print(not_null)  

not_null = not_null.drop_duplicates(subset=['ensg'])
print('dup removed expn number: ',len(not_null.index))

# add new column by abs(log2FoldChange)
not_null['abs_log2FoldChange']=abs (not_null['log2FoldChange'])
not_null['pvalue_with_tiny']=(not_null['pvalue'] + sys.float_info.min)

# sort by padj_with_tiny and fold_change
sorted_data = not_null.sort_values(by = ['pvalue_with_tiny', 'abs_log2FoldChange'], ascending = [True, False], na_position = 'last')

# drop unwanted columns
# unwanted = ['mean_A8301_2.0','err_A8301_2.0','err_B14_2.0','err_control_2.0','mean_Eli_2.0','err_Eli_2.0','err_TGFb_2.0'] 
# geneid_not_null = geneid_not_null.drop(unwanted,axis=1)
# not_null.to_excel("FOXA1_B14_ATACB14_k27ac.annotation.deg.ATACB14uniq.xls")
sorted_data.to_csv("test.%s" % 'B14_FOXA1.ATACUniq.K27ac-ann10000kb.withpvalue.tsv',sep='\t',index=False) 

print('zscore match finished!')   

# use expn's zscore to draw a volcano plot
# expn = expression(
#     filename='/Users/jplab/Desktop/2021-7/data/7-2/test.B14_FOXA1.ATACUniq.K27ac-ann10000kb.withpvalue.tsv', 
#     format={'force_tsv': True, 'ensg': 0, 'skiplines':0,'pvalue':20,'padj':21}, expn='[column[11], column[14]]',
#     cond_names=[
#     'mean_B14_2.0',
#     'mean_TGF_2.0',
#     ] 
# )

# print('expn',expn)

# print('serialisedArrayDataDict: ',expn.serialisedArrayDataDict)

# print('mean_B14_2.0','mean_TGF_2.0' in expn.serialisedArrayDataDict)

# print(expn.keys())

# expn.volcano(
#     condition_name='mean_B14_2.0',
#     p_value_key='pvalue',
#     filename='volcano_pdf'
#     )

# visuz.gene_exp.volcano(df=df, lfc='log2FC', pv='p-value', lfc_thr=(1, 2), pv_thr=(0.05, 0.01), 
#     color=("#00239CFF", "grey", "#E10600FF"), plotlegend=True, legendpos='upper right', 
#     legendanchor=(1.46,1))



dup removed expn number:  1719
zscore match finished!


In [84]:

volca = visuz.gene_exp.volcano(
    df=sorted_data, 
    lfc='log2FoldChange', 
    pv='pvalue_with_tiny',
    valpha=0.5,
    lfc_thr=(1, 1),
    pv_thr=(0.01, 0.01),
    sign_line=True,
    geneid="ensg", 
    genenames=({"ENSG00000106366":"SERPINE1","ENSG00000099994":"SUSD2","ENSG00000130513":"GDF15","ENSG00000129422":"MTUS1","ENSG00000115414":"FN1","ENSG00000124882":"EREG","ENSG00000011028":"MRC2","ENSG00000023839":"ABCC2","ENSG00000111912":"NCOA7","ENSG00000151632":"AKR1C2","ENSG00000175727":"MLXIP","ENSG00000198074":"AKR1B10","ENSG00000166949":"SMAD3","ENSG00000175115":"PACS1","ENSG00000170558":"CDH2","ENSG00000117984":"CTSD","ENSG00000205336":"ADGRG1","ENSG00000138448":"ITGAV","ENSG00000115884":"SDC1","ENSG00000153815":"CMIP","ENSG00000070614":"NDST1","ENSG00000196498":"NCOR2","ENSG00000134107":"BHLHE40","ENSG00000136205":"TNS3","ENSG00000161011":"SQSTM1","ENSG00000157168":"NRG1","ENSG00000101255":"TRIB3","ENSG00000143641":"GALNT2","ENSG00000117983":"MUC5B","ENSG00000008513":"ST3GAL1","ENSG00000129116":"PALLD","ENSG00000169604":"ANTXR1","ENSG00000272398":"CD24","ENSG00000059804":"SLC2A3","ENSG00000074527":"NTN4","ENSG00000131473":"ACLY","ENSG00000146674":"IGFBP3","ENSG00000114861":"FOXP1","ENSG00000127948":"POR","ENSG00000120875":"DUSP4","ENSG00000188064":"WNT7B","ENSG00000147010":"SH3KBP1","ENSG00000103335":"PIEZO1","ENSG00000142192":"APP","ENSG00000144063":"MALL","ENSG00000084731":"KIF3C","ENSG00000167460":"TPM4","ENSG00000118257":"NRP2","ENSG00000169297":"NR0B1"}),
    gstyle=2,
    gfont=3,
    # show=True,
    plotlegend=True,
    dim=(50,20)
    )

print('volcano plot done!!!')

volcano plot done!!!
